# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/990.0 kB ? eta -:--:--
     - ----------------------------------- 30.7/990.0 kB 660.6 kB/s eta 0:00:02
     - ----------------------------------- 41.0/990.0 kB 393.8 kB/s eta 0:00:03
     -- ---------------------------------- 71.7/990.0 kB 491.5 kB/s eta 0:00:02
     ----- ------------------------------ 153.6/990.0 kB 833.5 kB/s eta 0:00:02
     ------- ---------------------------- 194.6/990.0 kB 787.7 kB/s eta 0:00:02
     ------------ ------------------------- 317.4/990.0 kB 1.2 MB/s eta 0:00:01
     ------------ ------------------------- 317.4/990.0 kB 1.2 MB/s eta 0:00:01
     ---------------------- --------------- 583.7/990.0 kB 1.5 MB/s eta 0:00:01
     ---------------------- --------------- 583.7/990.0 kB 1.5 MB/s eta 0:00:01
     ------------------------- ------------ 665.6/990.0 kB 1.4 MB/s eta 0:00:01
     -------------------------- ----------- 696.3/990.0 kB 1.4 MB/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.7 requires openai<2.0.0,>=1.24.0, but you have openai 1.14.2 which is incompatible.


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [3]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [4]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


biology_template  = """你是一位经验丰富的生物学家。 
你对生物学的各个分支都有深入的了解，
特别擅长将复杂的生物学概念解释得通俗易懂。 
当面对你不熟悉的问题时，你会以科学的谨慎态度进行回答。

这是一个问题：
{input}"""


computer_scientist_template = """你是一位杰出的计算机学家。 
你对计算机科学的理论和实践都有深刻的洞察，
能够解决从算法到系统架构的各种问题。 
如果遇到超出你知识范围的问题，你会诚实地表示需要进一步研究。

这是一个问题： {input}"""

chinese_literature_scholar_template = """你是一位博学的汉语言文学家。 
你对中国的文学作品和语言学有着深厚的理解和独到的见解，
善于从历史和文化的角度分析和解读。 
当遇到难以回答的问题时，你会保持学者的谦逊，寻找更多的资料以提供准确的答案。

这是一个问题： {input}"""

In [5]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_scientist_template,
    },
        {
        "name": "汉语言",
        "description": "适用于回答汉语言问题",
        "prompt_template": chinese_literature_scholar_template,
    },
]

In [22]:
import os

llm = OpenAI(model_name="gpt-3.5-turbo-instruct",base_url= os.getenv("BASE_URL"))

In [7]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [8]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [9]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [10]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [11]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言: 适用于回答汉语言问题']


In [12]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言问题


In [13]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [14]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言问题

<< INPUT >>
{input}

<< OU

In [15]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [16]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指一种理想的物体，它可以吸收所有进入它的能量，无论是电磁波还是粒子，都会被完全吸收。同时，黑体也可以以最高效率辐射出所有吸收的能量，产生热辐射。根据黑体辐射的性质，我们可以推导出普朗克定律和斯蒂芬-玻尔兹曼定律等重要物理规律。'}


In [18]:
print(
    chain.invoke(
        "在细胞分裂过程中，有丝分裂和减数分裂有什么区别？"
    )
)



> Entering new MultiPromptChain chain...
生物: {'input': '在细胞分裂过程中，有丝分裂和减数分裂有什么区别？'}
> Finished chain.
{'input': '在细胞分裂过程中，有丝分裂和减数分裂有什么区别？', 'text': '\n\n丝分裂和减数分裂都是细胞分裂的两种方式，它们的主要区别在于细胞分裂的目的和产生的细胞数量。\n\n丝分裂是一种有性细胞分裂，它发生在有细胞核的生物体中，如动物、植物等。它的主要目的是产生两个完全相同的子细胞，这两个子细胞具有相同的染色体数目和遗传信息。丝分裂包括有丝分裂前期、有丝分裂中期、有丝分裂后期和有丝分裂结束期四个阶段，其中涉及到染色体的复制、排列和分离过程。\n\n减数分裂是一种有性细胞分裂，它发生在生殖细胞中，如'}


In [19]:
print(
    chain.invoke(
        "什么是面向对象编程（OOP）中的'继承'，它是如何有助于代码重用的？"
    )
)



> Entering new MultiPromptChain chain...
计算机: {'input': "什么是面向对象编程（OOP）中的'继承'，它是如何有助于代码重用的？"}
> Finished chain.
{'input': "什么是面向对象编程（OOP）中的'继承'，它是如何有助于代码重用的？", 'text': '\n\n继承是面向对象编程中的一种概念，它允许一个类（称为子类）继承另一个类（称为父类）的属性和方法。这意味着子类可以使用父类的属性和方法，而不需要重新编写相同的代码。这有助于代码重用，使得程序更加简洁和易于维护。\n\n举个例子，假设有一个父类叫做“动物”，它有一个属性叫做“体重”和一个方法叫做“移动”。现在如果我们想创建一个子类叫做“狗”，它继承了“动物”的属性和方法，那么我们就不需要重新定义“体重”属性和“移动”方法，因为它们已经存在于父类中。我们只需要在“狗”的类中添加一些特定的属性和方法，比如“种类”和“叫声”。\n\n'}


In [20]:
print(
    chain.invoke(
        "请解释汉语中的'四声'是什么，并给出每个声调的一个例字?"
    )
)



> Entering new MultiPromptChain chain...
汉语言: {'input': "请解释汉语中的'四声'是什么，并给出每个声调的一个例字?"}
> Finished chain.
{'input': "请解释汉语中的'四声'是什么，并给出每个声调的一个例字?", 'text': '\n\n汉语中的四声指的是声调，即在发音时声音的高低、升降和变化。它们分别是：平声、上声、去声和入声。\n平声是指声调平稳，没有升降变化，发音时声带保持松弛状态，声音持续稳定。例字：爸爸、妈妈。\n上声是指发音时先升高声调，再降低回到原来的声调，发音时声带紧张，声音明显上扬。例字：哥哥、姐姐。\n去声是指发音时先降低声调，再回升到原来的声调，发音时声带紧张，声音明显下降。例字：猫、狗。\n入声是指发音时先抑后扬，声调由低变高，发音时声带紧张，'}


In [17]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要确定大于40的质数。根据质数的定义，质数是只能被1和自身整除的数，\n因此我们可以从41开始，依次判断每个数是否为质数。\n\n41是质数，但41+1=42不能被3整除，因此我们继续往下寻找。\n\n43是质数，但43+1=44也不能被3整除。\n\n47是质数，但47+1=48能被3整除，因此我们得出结论：大于40的第一个质数是47，\n且47+1=48能被3整除。'}


In [17]:
router_chain.verbose = True

In [18]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种极端密度和强大引力的天体，它的重力场非常强大，甚至连光也无法逃脱。它的存在是由于恒星在死亡时发生了坍缩，使得它们的质量集中在极小的空间内。黑洞的存在已经被多项观测和理论证据所证实，但仍然有很多未解之谜。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains